In [51]:
import re
import os
import glob
import pandas as pd

In [3]:
def process_raw_data(data):
    data['Protein IDs'] = data['Protein IDs'].str.split(';').str[0]
    data = data[~data['Protein IDs'].str.contains('REV|CON')]
    return data

In [4]:
index_map = pd.read_csv('map_result/index_map.csv',index_col = 0)
index_map.drop_duplicates(subset = 'Protein IDs' , inplace = True)
index_map

,Protein IDs,Processed Protein IDs
0,1433B_HUMAN,P31946|1433B_HUMAN
1,1433B_MOUSE,Q9CQV8|1433B_MOUSE
2,1433E_HUMAN,P62258|1433E_HUMAN
3,1433E_MOUSE,P62259|1433E_MOUSE
4,1433F_HUMAN,Q04917|1433F_HUMAN
...,...,...
116415,sp|Q9Y6Y1-3|CMTA1_HUMANIsoform3ofCalmodulin-bi...,Q9Y6Y1-3|CMTA1_HUMAN
116417,sp|Q9Y6Y8-2|S23IP_HUMANIsoform2ofSEC23-interac...,Q9Y6Y8-2|S23IP_HUMAN
116420,sp|Q9Y6Z4|KIAS1_HUMANPutativeuncharacterizedpr...,Q9Y6Z4|KIAS1_HUMAN
116421,sp|Q9Y6Z5-2|MLAS1_HUMANIsoform2ofPutativeuncha...,Q9Y6Z5-2|MLAS1_HUMAN


## K562

In [30]:
data = pd.read_csv('MS-based-SCP/Peripheral blood/SCP95832/all_intensity_included_proteinGroups_DDA.txt.csv',index_col = 0)
data = process_raw_data(data)
data = data[[col for col in data.columns if 'LFQ' not in col and 'Reporter' not in col]]
data['Protein IDs'] = data['Protein IDs'].str.replace(r'^sp\|','',regex=True)
data = data.rename(columns={'Protein IDs':'Processed Protein IDs'})
data.set_index('Processed Protein IDs',inplace =True)
data = data[data.index.str.endswith('HUMAN')]
data.columns = data.columns + '_SCP95832' + '_PeripheralBlood' + '_Human' + '_K562'
data

,Intensity_SCP95832_PeripheralBlood_Human_K562,Intensity 1_Exploris_RSLC9_Waters_CC_DDA_M2_1_SCP95832_PeripheralBlood_Human_K562,Intensity 1_Exploris_RSLC9_Waters_CC_DDA_M2_5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_0c5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_1_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_10_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M2_0c5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M2_10_SCP95832_PeripheralBlood_Human_K562,Intensity 4_Exploris_RSLC9_Waters_CC_DDA_M3_0c5_SCP95832_PeripheralBlood_Human_K562,Intensity 4_Exploris_RSLC9_Waters_CC_DDA_M3_10_SCP95832_PeripheralBlood_Human_K562,Intensity 4_Exploris_RSLC9_Waters_CC_DDA_M3_5_SCP95832_PeripheralBlood_Human_K562,Intensity 6_Exploris_RSLC9_Waters_CC_DDA_M3_1_SCP95832_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,,,,,,
Q9NZJ9-3|NUDT4_HUMAN,4447700.0,0,0.0,0,0,2711500.0,0.0,0,1736300.0,0,0.0,0,0
P0DPI2|GAL3A_HUMAN,13536000.0,0,1009600.0,0,0,2700800.0,0.0,0,2118200.0,0,7707500.0,0,0
Q9ULC4-2|MCTS1_HUMAN,4022300.0,0,642990.0,0,0,2052600.0,0.0,0,1326700.0,0,0.0,0,0
A0AVT1-2|UBA6_HUMAN,4017700.0,0,0.0,0,0,1153400.0,0.0,0,1734200.0,166870,963230.0,0,0
A0MZ66-2|SHOT1_HUMAN,4007600.0,0,0.0,0,0,2199000.0,0.0,0,648720.0,0,1159900.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y678|COPG1_HUMAN,20034000.0,0,4187600.0,0,0,4919200.0,761430.0,159030,8266300.0,88425,1651800.0,0,0
Q9Y696|CLIC4_HUMAN,17538000.0,0,3703500.0,0,91148,5003300.0,1670100.0,0,3748300.0,272660,2375200.0,561720,112220
Q9Y6A5|TACC3_HUMAN,9640600.0,0,861070.0,0,0,2431200.0,1147200.0,0,3309700.0,198120,1693400.0,0,0


In [31]:
data2 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP52016/all_intensity_included_proteinGroups-AML-singleCell.txt.csv',index_col = 0)
data2 = process_raw_data(data2)
data2 = data2[[col for col in data2.columns if 'Reporter' not in col and 'LFQ' not in col]]
data2 = data2.merge(
    index_map[['Protein IDs','Processed Protein IDs']],
    on = 'Protein IDs',
    how = 'left'
)
data2.drop('Protein IDs',axis = 1,inplace = True)
data2.set_index('Processed Protein IDs',inplace = True)
data2 = data2[data2.index.notna()]
data2 = data2[data2.index.str.endswith('HUMAN')]
data2.columns = data2.columns + '_SCP52016' + '_PeripheralBlood' + '_Human' + '_K562'
data2

,Intensity_SCP52016_PeripheralBlood_Human_K562,Intensity 1_SCP52016_PeripheralBlood_Human_K562,Intensity 10_SCP52016_PeripheralBlood_Human_K562,Intensity 11_SCP52016_PeripheralBlood_Human_K562,Intensity 12_SCP52016_PeripheralBlood_Human_K562,Intensity 13_SCP52016_PeripheralBlood_Human_K562,Intensity 2_SCP52016_PeripheralBlood_Human_K562,Intensity 3_SCP52016_PeripheralBlood_Human_K562,Intensity 4_SCP52016_PeripheralBlood_Human_K562,Intensity 5_SCP52016_PeripheralBlood_Human_K562,Intensity 6_SCP52016_PeripheralBlood_Human_K562,Intensity 7_SCP52016_PeripheralBlood_Human_K562,Intensity 8_SCP52016_PeripheralBlood_Human_K562,Intensity 9_SCP52016_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,,,,,,,
P31946|1433B_HUMAN,2108200000,281640000,81871000,142390000,180560000,194730000,184260000,209030000,134980000,180070000,81424000,78589000,223310000,135390000
P62258|1433E_HUMAN,11107000000,749010000,850010000,885960000,826590000,1060700000,763250000,828950000,753630000,850950000,927590000,917450000,832190000,861080000
Q04917|1433F_HUMAN,893780000,60218000,76181000,69418000,63391000,70889000,65313000,73395000,72191000,59568000,70677000,69312000,65254000,77974000
P61981|1433G_HUMAN,2451700000,191600000,193200000,188360000,178270000,195620000,198640000,197080000,190980000,227280000,167520000,156890000,175600000,190670000
P31947|1433S_HUMAN,878740000,63572000,68853000,70822000,68302000,71282000,62891000,66586000,61973000,61376000,73303000,76295000,70020000,63467000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q14966|ZN638_HUMAN,2524600,0,0,2524600,0,0,0,0,0,0,0,0,0,0
O75312|ZPR1_HUMAN,68644000,5581800,8984200,5358800,0,0,5622000,3238200,6078100,3964200,6002200,6920500,7792600,9101700
O95218|ZRAB2_HUMAN,95336000,5554300,13847000,13142000,29132000,12188000,0,0,0,0,0,5113100,8972900,7386800


In [17]:
# SCP78559和SCP52016是一套数据
data3 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP78559/all_intensity_included_proteinGroups-AML-singleCell.txt.csv',index_col = 0)
data3 = process_raw_data(data3)
data3 = data3[[col for col in data3.columns if 'Reporter' not in col and 'LFQ' not in col]]
data3 = data3.merge(
    index_map[['Protein IDs','Processed Protein IDs']],
    on = 'Protein IDs',
    how = 'left'
)
data3.drop('Protein IDs',axis = 1,inplace = True)
data3.set_index('Processed Protein IDs',inplace = True)
data3 = data3[data3.index.notna()]
data3 = data3[data3.index.str.endswith('HUMAN')]
data3.columns = data3.columns + '_SCP78559' + '_PeripheralBlood' + '_Human' + '_K562'
data3

,Intensity_SCP78559_PeripheralBlood_Human_K562,Intensity 1_SCP78559_PeripheralBlood_Human_K562,Intensity 10_SCP78559_PeripheralBlood_Human_K562,Intensity 11_SCP78559_PeripheralBlood_Human_K562,Intensity 12_SCP78559_PeripheralBlood_Human_K562,Intensity 13_SCP78559_PeripheralBlood_Human_K562,Intensity 2_SCP78559_PeripheralBlood_Human_K562,Intensity 3_SCP78559_PeripheralBlood_Human_K562,Intensity 4_SCP78559_PeripheralBlood_Human_K562,Intensity 5_SCP78559_PeripheralBlood_Human_K562,Intensity 6_SCP78559_PeripheralBlood_Human_K562,Intensity 7_SCP78559_PeripheralBlood_Human_K562,Intensity 8_SCP78559_PeripheralBlood_Human_K562,Intensity 9_SCP78559_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,,,,,,,
P31946|1433B_HUMAN,2108200000,281640000,81871000,142390000,180560000,194730000,184260000,209030000,134980000,180070000,81424000,78589000,223310000,135390000
P62258|1433E_HUMAN,11107000000,749010000,850010000,885960000,826590000,1060700000,763250000,828950000,753630000,850950000,927590000,917450000,832190000,861080000
Q04917|1433F_HUMAN,893780000,60218000,76181000,69418000,63391000,70889000,65313000,73395000,72191000,59568000,70677000,69312000,65254000,77974000
P61981|1433G_HUMAN,2451700000,191600000,193200000,188360000,178270000,195620000,198640000,197080000,190980000,227280000,167520000,156890000,175600000,190670000
P31947|1433S_HUMAN,878740000,63572000,68853000,70822000,68302000,71282000,62891000,66586000,61973000,61376000,73303000,76295000,70020000,63467000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q14966|ZN638_HUMAN,2524600,0,0,2524600,0,0,0,0,0,0,0,0,0,0
O75312|ZPR1_HUMAN,68644000,5581800,8984200,5358800,0,0,5622000,3238200,6078100,3964200,6002200,6920500,7792600,9101700
O95218|ZRAB2_HUMAN,95336000,5554300,13847000,13142000,29132000,12188000,0,0,0,0,0,5113100,8972900,7386800


In [18]:
# SCP61427 和上述一套数据
data4 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP61427/all_intensity_included_proteinGroups-AML-singleCell.txt.csv',index_col = 0)
data4 = process_raw_data(data4)
data4

,Protein IDs,Reporter intensity corrected 1,Reporter intensity corrected 2,Reporter intensity corrected 3,Reporter intensity corrected 4,Reporter intensity corrected 5,Reporter intensity corrected 6,Reporter intensity corrected 7,Reporter intensity corrected 8,Reporter intensity corrected 9,...,Intensity 12,Intensity 13,Intensity 2,Intensity 3,Intensity 4,Intensity 5,Intensity 6,Intensity 7,Intensity 8,Intensity 9
0,1433B_HUMAN,14237000.0,784970.0,206140.0,38029.0,75054.00,74527.00,51397.0,57480.0,84755.0,...,180560000,194730000,184260000,209030000,134980000,180070000,81424000,78589000,223310000,135390000
1,1433E_HUMAN,110730000.0,6006600.0,580240.0,177940.0,892400.00,658160.00,532200.0,660410.0,594050.0,...,826590000,1060700000,763250000,828950000,753630000,850950000,927590000,917450000,832190000,861080000
2,1433F_HUMAN,14403000.0,787730.0,181520.0,32517.0,92619.00,58864.00,47054.0,58586.0,72022.0,...,63391000,70889000,65313000,73395000,72191000,59568000,70677000,69312000,65254000,77974000
3,1433G_HUMAN,30719000.0,1665300.0,254660.0,58282.0,215660.00,125550.00,97913.0,120040.0,152120.0,...,178270000,195620000,198640000,197080000,190980000,227280000,167520000,156890000,175600000,190670000
4,1433S_HUMAN,11109000.0,587520.0,0.0,2190.4,79981.00,58721.00,30408.0,54486.0,68060.0,...,68302000,71282000,62891000,66586000,61973000,61376000,73303000,76295000,70020000,63467000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2682,ZN638_HUMAN,85273.0,5224.5,7527.5,1187.8,308.75,712.64,350.0,1116.6,1005.4,...,0,0,0,0,0,0,0,0,0,0
2683,ZPR1_HUMAN,1479800.0,79067.0,16810.0,4720.3,14466.00,11900.00,12880.0,10960.0,13013.0,...,0,0,5622000,3238200,6078100,3964200,6002200,6920500,7792600,9101700
2684,ZRAB2_HUMAN,2670300.0,147550.0,28044.0,5586.5,14167.00,10362.00,6196.1,8419.5,15148.0,...,29132000,12188000,0,0,0,0,0,5113100,8972900,7386800
2685,ZW10_HUMAN,586350.0,30159.0,17277.0,4242.5,7690.60,5945.10,5969.9,4833.9,6591.7,...,0,0,5707800,1891400,2046800,2120700,7001400,1773900,5937000,4829000


In [32]:
data5 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP34196/all_intensity_included_ProteinGroups_AML_SingleCell.txt.csv',index_col = 0)
data5 = process_raw_data(data5)
data5 = data5[[col for col in data5.columns if 'Reporter' not in col and 'LFQ' not in col]]
data5 = data5.merge(
    index_map[['Protein IDs','Processed Protein IDs']],
    on = 'Protein IDs',
    how = 'left'
)
data5.drop('Protein IDs',axis = 1,inplace = True)
data5.set_index('Processed Protein IDs',inplace = True)
data5 = data5[data5.index.notna()]
data5 = data5[data5.index.str.endswith('HUMAN')]
data5.columns = data5.columns + '_SCP34196' + '_PeripheralBlood' + '_Human' + '_K562'
data5

,Intensity_SCP34196_PeripheralBlood_Human_K562,Intensity A1_SCP34196_PeripheralBlood_Human_K562,Intensity A2_SCP34196_PeripheralBlood_Human_K562,Intensity A3_SCP34196_PeripheralBlood_Human_K562,Intensity A4_SCP34196_PeripheralBlood_Human_K562,Intensity A5_SCP34196_PeripheralBlood_Human_K562,Intensity A6_SCP34196_PeripheralBlood_Human_K562,Intensity A7_SCP34196_PeripheralBlood_Human_K562,Intensity B1_SCP34196_PeripheralBlood_Human_K562,Intensity B2_SCP34196_PeripheralBlood_Human_K562,Intensity B3_SCP34196_PeripheralBlood_Human_K562,Intensity B4_SCP34196_PeripheralBlood_Human_K562,Intensity B5_SCP34196_PeripheralBlood_Human_K562,Intensity B6_SCP34196_PeripheralBlood_Human_K562,Intensity B7_SCP34196_PeripheralBlood_Human_K562,Intensity B8_SCP34196_PeripheralBlood_Human_K562,Intensity C1_SCP34196_PeripheralBlood_Human_K562,Intensity C2_SCP34196_PeripheralBlood_Human_K562,Intensity C3_SCP34196_PeripheralBlood_Human_K562,Intensity C4_SCP34196_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,,,,,,,,,,,,,
P31946|1433B_HUMAN,3641900000,248330000,196910000,224830000,248640000,139940000,257930000,193860000,94356000,184570000,246990000,148330000,164610000,155770000,274550000,212360000,186830000,177690000,117390000,167990000
P62258|1433E_HUMAN,14895000000,731830000,747080000,765970000,741240000,707150000,777400000,834340000,874630000,869360000,781900000,822650000,788220000,857780000,817820000,1004000000,728680000,677370000,732350000,635170000
Q04917|1433F_HUMAN,1025100000,49673000,50184000,62777000,64728000,47065000,58789000,59594000,57941000,56032000,58373000,51429000,63822000,59260000,58895000,62486000,46794000,38057000,37102000,42073000
P61981|1433G_HUMAN,3187400000,155670000,143580000,168480000,157010000,163940000,164750000,165670000,165830000,165470000,157970000,172640000,167990000,164320000,179560000,191070000,205600000,153570000,175320000,168990000
P31947|1433S_HUMAN,1224200000,63572000,62891000,66586000,61973000,62171000,66165000,61376000,73504000,76295000,70020000,63467000,68853000,70822000,68302000,71282000,59280000,52910000,52780000,51993000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P17017|ZNF14_HUMAN,32717000,2336100,2298000,2578400,2343300,2283100,2411300,2349000,2866300,0,0,0,0,2244700,3084900,3054800,0,0,2477500,2389900
O75312|ZPR1_HUMAN,122730000,5487100,5622000,5131200,6078100,5581600,4988600,6233200,6002200,6920500,7792600,9101700,8984200,7565800,7514700,2898700,6376700,5822000,7001100,7623400
O95218|ZRAB2_HUMAN,280400000,17880000,16564000,17063000,10814000,17617000,13389000,7721800,15647000,13778000,17402000,19491000,18415000,12152000,19556000,18586000,13654000,13016000,10058000,7593900


In [33]:
data6 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP95693/all_intensity_included_proteinGroups_all.txt.csv',index_col = 0)
data6 = process_raw_data(data6)
data6 = data6[[col for col in data6.columns if 'LFQ' not in col]]
data6 = data6.merge(
    index_map[['Protein IDs','Processed Protein IDs']],
    on = 'Protein IDs',
    how = 'left'
)
data6.drop('Protein IDs',axis = 1,inplace = True)
data6.set_index('Processed Protein IDs',inplace = True)
data6 = data6[data6.index.notna()]
data6 = data6[data6.index.str.endswith('HUMAN')]
data6.columns = data6.columns + '_SCP95693' + '_PeripheralBlood' + '_Human' + '_K562'
data6

,Intensity_SCP95693_PeripheralBlood_Human_K562,Intensity 20 HeLa cells_SCP95693_PeripheralBlood_Human_K562,Intensity Blank_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 1_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 2_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 4_SCP95693_PeripheralBlood_Human_K562,Intensity 100 HeLa cells_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 3_SCP95693_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,
A0AVT1|UBA6_HUMAN,2628200,0,0.0,0.0,0.0,0.0,2628200,0
A6NDG6|PGP_HUMAN,2358500,411310,0.0,0.0,0.0,97001.0,1824800,25408
P30046|DOPD_HUMAN,9721700,1082100,0.0,0.0,0.0,82810.0,8556700,0
A6NHQ2|FBLL1_HUMAN,2340700,0,0.0,0.0,0.0,0.0,2340700,0
A6NKF1|SAC31_HUMAN,19183000,0,0.0,0.0,0.0,0.0,19183000,0
...,...,...,...,...,...,...,...,...
Q9Y6C9|MTCH2_HUMAN,6248200,816390,0.0,26491.0,27397.0,279900.0,5098000,0
Q9Y6E2|5MP1_HUMAN,15330000,1385800,0.0,44789.0,70630.0,235730.0,13593000,0
Q9Y6G9|DC1L1_HUMAN,852270,339320,0.0,0.0,0.0,0.0,512950,0


In [26]:
# 和SCP34196 是同一套数据
data7 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP58167/all_intensity_included_ProteinGroups_AML_SingleCell.txt.csv',index_col = 0)
data7 = process_raw_data(data7)
data7

,Protein IDs,Reporter intensity corrected 1 A1,Reporter intensity corrected 2 A1,Reporter intensity corrected 3 A1,Reporter intensity corrected 4 A1,Reporter intensity corrected 5 A1,Reporter intensity corrected 6 A1,Reporter intensity corrected 7 A1,Reporter intensity corrected 8 A1,Reporter intensity corrected 9 A1,...,Intensity B3,Intensity B4,Intensity B5,Intensity B6,Intensity B7,Intensity B8,Intensity C1,Intensity C2,Intensity C3,Intensity C4
0,1433B_HUMAN,19.279,1,0.30972,0.093235,0.09051,0.111390,0.059335,0.064355,0.148770,...,246990000,148330000,164610000,155770000,274550000,212360000,186830000,177690000,117390000,167990000
1,1433E_HUMAN,17.930,1,0.55336,0.056829,0.19594,0.119530,0.081480,0.103990,0.108670,...,781900000,822650000,788220000,857780000,817820000,1004000000,728680000,677370000,732350000,635170000
2,1433F_HUMAN,17.884,1,0.50388,0.040414,0.17540,0.091582,0.069903,0.136010,0.124260,...,58373000,51429000,63822000,59260000,58895000,62486000,46794000,38057000,37102000,42073000
3,1433G_HUMAN,18.359,1,0.69917,0.073671,0.15628,0.114290,0.059684,0.070884,0.089428,...,157970000,172640000,167990000,164320000,179560000,191070000,205600000,153570000,175320000,168990000
4,1433S_HUMAN,18.939,1,0.00000,0.000000,0.15485,0.100910,0.049140,0.063019,0.097130,...,70020000,63467000,68853000,70822000,68302000,71282000,59280000,52910000,52780000,51993000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,ZNF14_HUMAN,0.000,0,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,2244700,3084900,3054800,0,0,2477500,2389900
2642,ZPR1_HUMAN,19.370,1,0.80069,0.067863,0.20994,0.182160,0.152430,0.078451,0.127250,...,7792600,9101700,8984200,7565800,7514700,2898700,6376700,5822000,7001100,7623400
2643,ZRAB2_HUMAN,15.925,1,0.42514,0.023356,0.16979,0.093636,0.073304,0.116440,0.088761,...,17402000,19491000,18415000,12152000,19556000,18586000,13654000,13016000,10058000,7593900
2644,ZW10_HUMAN,17.836,1,0.55140,0.204320,0.27533,0.333410,0.139210,0.075740,0.114180,...,4943500,1716400,4570200,6246600,6161500,4905900,7059500,7178700,1580800,6623200


In [27]:
# 和上面是同一套数据
data8 = pd.read_csv('MS-based-SCP/Peripheral blood/SCP56289/all_intensity_included_ProteinGroups_AML_SingleCell.txt.csv',index_col = 0)
data8 = process_raw_data(data8)
data8

,Protein IDs,Reporter intensity corrected 1 A1,Reporter intensity corrected 2 A1,Reporter intensity corrected 3 A1,Reporter intensity corrected 4 A1,Reporter intensity corrected 5 A1,Reporter intensity corrected 6 A1,Reporter intensity corrected 7 A1,Reporter intensity corrected 8 A1,Reporter intensity corrected 9 A1,...,Intensity B3,Intensity B4,Intensity B5,Intensity B6,Intensity B7,Intensity B8,Intensity C1,Intensity C2,Intensity C3,Intensity C4
0,1433B_HUMAN,19.279,1,0.30972,0.093235,0.09051,0.111390,0.059335,0.064355,0.148770,...,246990000,148330000,164610000,155770000,274550000,212360000,186830000,177690000,117390000,167990000
1,1433E_HUMAN,17.930,1,0.55336,0.056829,0.19594,0.119530,0.081480,0.103990,0.108670,...,781900000,822650000,788220000,857780000,817820000,1004000000,728680000,677370000,732350000,635170000
2,1433F_HUMAN,17.884,1,0.50388,0.040414,0.17540,0.091582,0.069903,0.136010,0.124260,...,58373000,51429000,63822000,59260000,58895000,62486000,46794000,38057000,37102000,42073000
3,1433G_HUMAN,18.359,1,0.69917,0.073671,0.15628,0.114290,0.059684,0.070884,0.089428,...,157970000,172640000,167990000,164320000,179560000,191070000,205600000,153570000,175320000,168990000
4,1433S_HUMAN,18.939,1,0.00000,0.000000,0.15485,0.100910,0.049140,0.063019,0.097130,...,70020000,63467000,68853000,70822000,68302000,71282000,59280000,52910000,52780000,51993000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,ZNF14_HUMAN,0.000,0,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,2244700,3084900,3054800,0,0,2477500,2389900
2642,ZPR1_HUMAN,19.370,1,0.80069,0.067863,0.20994,0.182160,0.152430,0.078451,0.127250,...,7792600,9101700,8984200,7565800,7514700,2898700,6376700,5822000,7001100,7623400
2643,ZRAB2_HUMAN,15.925,1,0.42514,0.023356,0.16979,0.093636,0.073304,0.116440,0.088761,...,17402000,19491000,18415000,12152000,19556000,18586000,13654000,13016000,10058000,7593900
2644,ZW10_HUMAN,17.836,1,0.55140,0.204320,0.27533,0.333410,0.139210,0.075740,0.114180,...,4943500,1716400,4570200,6246600,6161500,4905900,7059500,7178700,1580800,6623200


In [34]:
merged_data = pd.concat([data, data2, data5, data6], axis=0, join='outer')
final_data = merged_data.groupby(merged_data.index).first()
final_data.to_csv('MS-based-SCP/A_Checked_data/Imumune_Cell_Line/PeripheralBlood_Human_K562.csv')
final_data

,Intensity_SCP95832_PeripheralBlood_Human_K562,Intensity 1_Exploris_RSLC9_Waters_CC_DDA_M2_1_SCP95832_PeripheralBlood_Human_K562,Intensity 1_Exploris_RSLC9_Waters_CC_DDA_M2_5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_0c5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_1_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_10_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M1_5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M2_0c5_SCP95832_PeripheralBlood_Human_K562,Intensity 2_Exploris_RSLC9_Waters_CC_DDA_M2_10_SCP95832_PeripheralBlood_Human_K562,Intensity 4_Exploris_RSLC9_Waters_CC_DDA_M3_0c5_SCP95832_PeripheralBlood_Human_K562,...,Intensity C3_SCP34196_PeripheralBlood_Human_K562,Intensity C4_SCP34196_PeripheralBlood_Human_K562,Intensity_SCP95693_PeripheralBlood_Human_K562,Intensity 20 HeLa cells_SCP95693_PeripheralBlood_Human_K562,Intensity Blank_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 1_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 2_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 4_SCP95693_PeripheralBlood_Human_K562,Intensity 100 HeLa cells_SCP95693_PeripheralBlood_Human_K562,Intensity Single cell 3_SCP95693_PeripheralBlood_Human_K562
Processed Protein IDs,,,,,,,,,,,,,,,,,,,,,
A0AVT1-2|UBA6_HUMAN,4017700.0,0.0,0.0,0.0,0.0,1153400.0,0.0,0.0,1734200.0,166870.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0AVT1|UBA6_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15474000.0,13409000.0,2628200.0,0.0,0.0,0.0,0.0,0.0,2628200.0,0.0
A0FGR8|ESYT2_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4026900.0,4306700.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0MZ66-2|SHOT1_HUMAN,4007600.0,0.0,0.0,0.0,0.0,2199000.0,0.0,0.0,648720.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A3KN83|SBNO1_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1556700.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y6M9|NDUB9_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q9Y6V0|PCLO_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,13168000.0,0.0,0.0,0.0,0.0,0.0,13168000.0,0.0
Q9Y6V7|DDX49_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## U937

In [36]:
# 和SCP95693 K562是同一套数据
data = pd.read_csv('MS-based-SCP/Pleural effusion/SCP76140/all_intensity_included_proteinGroups_all.txt.csv',index_col = 0)
data = process_raw_data(data)
data = data[[col for col in data.columns if 'LFQ' not in col]]
data = data.merge(
    index_map[['Protein IDs','Processed Protein IDs']],
    on = 'Protein IDs',
    how = 'left'
)
data.drop('Protein IDs',axis = 1,inplace = True)
data.set_index('Processed Protein IDs',inplace = True)
data = data[data.index.notna()]
data = data[data.index.str.endswith('HUMAN')]
data.columns = data.columns + '_SCP76140' + '_PleuralEffusion' + '_Human' + '_U937'
data

,Intensity_SCP76140_PeripheralBlood_Human_U937,Intensity 20 HeLa cells_SCP76140_PeripheralBlood_Human_U937,Intensity Blank_SCP76140_PeripheralBlood_Human_U937,Intensity Single cell 1_SCP76140_PeripheralBlood_Human_U937,Intensity Single cell 2_SCP76140_PeripheralBlood_Human_U937,Intensity Single cell 4_SCP76140_PeripheralBlood_Human_U937,Intensity 100 HeLa cells_SCP76140_PeripheralBlood_Human_U937,Intensity Single cell 3_SCP76140_PeripheralBlood_Human_U937
Processed Protein IDs,,,,,,,,
A0AVT1|UBA6_HUMAN,2628200,0,0.0,0.0,0.0,0.0,2628200,0
A6NDG6|PGP_HUMAN,2358500,411310,0.0,0.0,0.0,97001.0,1824800,25408
P30046|DOPD_HUMAN,9721700,1082100,0.0,0.0,0.0,82810.0,8556700,0
A6NHQ2|FBLL1_HUMAN,2340700,0,0.0,0.0,0.0,0.0,2340700,0
A6NKF1|SAC31_HUMAN,19183000,0,0.0,0.0,0.0,0.0,19183000,0
...,...,...,...,...,...,...,...,...
Q9Y6C9|MTCH2_HUMAN,6248200,816390,0.0,26491.0,27397.0,279900.0,5098000,0
Q9Y6E2|5MP1_HUMAN,15330000,1385800,0.0,44789.0,70630.0,235730.0,13593000,0
Q9Y6G9|DC1L1_HUMAN,852270,339320,0.0,0.0,0.0,0.0,512950,0


In [40]:
data2 = pd.read_csv('MS-based-SCP/Pleural effusion/SCP47204/all_intensity_included_proteinGroups_hek_u937.txt.csv',index_col = 0)
data2 = process_raw_data(data2)
data2 = data2[[col for col in data2.columns if 'LFQ' not in col and 'Reporter' not in col]]
data2['Protein IDs'] = data2['Protein IDs'].str.replace(r'^sp\|','',regex=True)
data2 = data2.rename(columns={'Protein IDs':'Processed Protein IDs'})
data2.set_index('Processed Protein IDs',inplace =True)
data2 = data2[data2.index.str.endswith('HUMAN')]
data2.columns = data2.columns + '_SCP47204' + '_PleuralEffusion' + '_Human' + '_U937'
data2

,Intensity_SCP47204_PleuralEffusion_Human_U937,Intensity A_SCP47204_PleuralEffusion_Human_U937,Intensity B_SCP47204_PleuralEffusion_Human_U937,Intensity C_SCP47204_PleuralEffusion_Human_U937,Intensity D_SCP47204_PleuralEffusion_Human_U937,Intensity E_SCP47204_PleuralEffusion_Human_U937,Intensity F_SCP47204_PleuralEffusion_Human_U937,Intensity G_SCP47204_PleuralEffusion_Human_U937,Intensity H_SCP47204_PleuralEffusion_Human_U937,Intensity I_SCP47204_PleuralEffusion_Human_U937,Intensity J_SCP47204_PleuralEffusion_Human_U937,Intensity K_SCP47204_PleuralEffusion_Human_U937,Intensity L_SCP47204_PleuralEffusion_Human_U937,Intensity M_SCP47204_PleuralEffusion_Human_U937,Intensity N_SCP47204_PleuralEffusion_Human_U937,Intensity O_SCP47204_PleuralEffusion_Human_U937,Intensity P_SCP47204_PleuralEffusion_Human_U937
Processed Protein IDs,,,,,,,,,,,,,,,,,
P02538|K2C6A_HUMAN,2.092200e+09,1.882600e+09,24523000.0,0,0,0.0,0,0.0,0.0,0,0,1.152300e+08,0,0.0,0,0,69881000
P07477|TRY1_HUMAN,5.349900e+09,0.000000e+00,598950000.0,322320000,436070000,274760000.0,272480000,360160000.0,232750000.0,415970000,890140000,2.269700e+08,441030000,202580000.0,362840000,0,312840000
P08779|K1C16_HUMAN,6.818500e+09,5.130800e+09,65482000.0,0,0,0.0,0,0.0,21675000.0,0,0,1.527800e+09,0,0.0,0,2243300,70518000
P12035|K2C3_HUMAN,4.591200e+06,0.000000e+00,0.0,0,0,0.0,0,0.0,0.0,0,0,4.591200e+06,0,0.0,0,0,0
P13647|K2C5_HUMAN,2.190700e+09,1.371300e+09,17616000.0,0,0,0.0,0,0.0,0.0,0,0,7.843100e+08,0,0.0,0,0,17475000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y5S9|RBM8A_HUMAN,2.514800e+07,0.000000e+00,0.0,3427300,3483500,1839100.0,2028200,2869200.0,2345600.0,3752600,1940000,1.621800e+06,0,0.0,1840900,0,0
Q9Y617|SERC_HUMAN,2.666200e+06,0.000000e+00,0.0,1273400,1392800,0.0,0,0.0,0.0,0,0,0.000000e+00,0,0.0,0,0,0
Q9Y678|COPG1_HUMAN,2.272000e+06,0.000000e+00,0.0,0,0,0.0,0,0.0,0.0,2272000,0,0.000000e+00,0,0.0,0,0,0


In [41]:
#  SCP50571没有数据

In [44]:
data3 = pd.read_csv('MS-based-SCP/Pleural effusion/SCP94901/all_intensity_included_proteinGroups_Carrier_level_AGC.txt.csv',index_col = 0)
data3 = process_raw_data(data3)
data3 = data3[[col for col in data3.columns if 'LFQ' not in col and 'Reporter' not in col]]
data3['Protein IDs'] = data3['Protein IDs'].str.replace(r'^sp\|','',regex=True)
data3 = data3.rename(columns={'Protein IDs':'Processed Protein IDs'})
data3.set_index('Processed Protein IDs',inplace =True)
data3 = data3[data3.index.str.endswith('HUMAN')]
data3.columns = data3.columns + '_SCP94901' + '_CarrierLevelAGC' + '_PleuralEffusion' + '_Human' + '_U937'
data3

,Intensity_SCP94901_PleuralEffusion_Human_U937,Intensity 0041_SCP94901_PleuralEffusion_Human_U937,Intensity 0042_SCP94901_PleuralEffusion_Human_U937,Intensity 0043_SCP94901_PleuralEffusion_Human_U937,Intensity 0044_SCP94901_PleuralEffusion_Human_U937,Intensity 0045_SCP94901_PleuralEffusion_Human_U937,Intensity 0046_SCP94901_PleuralEffusion_Human_U937,Intensity 0100_SCP94901_PleuralEffusion_Human_U937,Intensity 0101_SCP94901_PleuralEffusion_Human_U937,Intensity 0102_SCP94901_PleuralEffusion_Human_U937,Intensity 0103_SCP94901_PleuralEffusion_Human_U937,Intensity 0104_SCP94901_PleuralEffusion_Human_U937,Intensity 0105_SCP94901_PleuralEffusion_Human_U937
Processed Protein IDs,,,,,,,,,,,,,
P07951-3|TPM2_HUMAN,1.510900e+08,3107200.0,14115000.0,7649500.0,24750000.0,5.373100e+07,4.773200e+07,0.0,0.0,0.0,0.0,0.0,0.0
P0DPI2-2|GAL3A_HUMAN,3.645600e+07,2793800.0,0.0,0.0,21466000.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,12195000.0
A0FGR8-2|ESYT2_HUMAN,1.565400e+07,0.0,0.0,0.0,3763100.0,6.434300e+06,5.456900e+06,0.0,0.0,0.0,0.0,0.0,0.0
Q14839|CHD4_HUMAN,2.053700e+07,0.0,0.0,0.0,0.0,0.000000e+00,1.170600e+07,0.0,0.0,0.0,2682800.0,0.0,6149100.0
P09651-3|ROA1_HUMAN,7.479100e+09,52166000.0,321790000.0,184040000.0,646010000.0,1.424800e+09,2.596400e+09,55986000.0,318500000.0,131800000.0,327240000.0,624050000.0,796340000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9Y617|SERC_HUMAN,2.772200e+08,0.0,5964900.0,10117000.0,28477000.0,1.100100e+08,0.000000e+00,7581000.0,23569000.0,10645000.0,26064000.0,37053000.0,17736000.0
Q9Y666|S12A7_HUMAN,2.933900e+06,0.0,0.0,0.0,0.0,0.000000e+00,2.933900e+06,0.0,0.0,0.0,0.0,0.0,0.0
Q9Y678|COPG1_HUMAN,2.232200e+07,0.0,0.0,0.0,4863900.0,6.491400e+06,1.096700e+07,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
data4 = pd.read_csv('MS-based-SCP/Pleural effusion/SCP94901/all_intensity_included_proteinGroups_Max_Fill_time.txt.csv',index_col = 0)
data4 = process_raw_data(data4)
data4 = data4[[col for col in data4.columns if 'LFQ' not in col and 'Reporter' not in col]]
data4['Protein IDs'] = data4['Protein IDs'].str.replace(r'^sp\|','',regex=True)
data4 = data4.rename(columns={'Protein IDs':'Processed Protein IDs'})
data4.set_index('Processed Protein IDs',inplace =True)
data4 = data4[data4.index.str.endswith('HUMAN')]
data4.columns = data4.columns + '_SCP94901' + '_MaxFillTime' +'_PleuralEffusion' + '_Human' + '_U937'
data4

,Intensity_SCP94901_MaxFillTime_PleuralEffusion_Human_U937
Processed Protein IDs,
O95932-2|TGM3L_HUMAN,0
P02790|HEMO_HUMAN,0
P0DV60|LCE7A_HUMAN,0
P23276|KELL_HUMAN,0
P26641|EF1G_HUMAN,5593300
P38919|IF4A3_HUMAN,3272800
P51861|CDR1_HUMAN,0
Q01804-5|OTUD4_HUMAN,2084000
Q14690|RRP5_HUMAN,0


In [49]:
data5 = pd.read_csv('MS-based-SCP/Pleural effusion/SCP47730/all_intensity_included_proteinGroups_FIG1.txt.csv',index_col = 0)
data5 = process_raw_data(data5)
data5 = data5[[col for col in data5.columns if 'LFQ' not in col and 'Reporter' not in col]]
data5['Protein IDs'] = data5['Protein IDs'].str.replace(r'^sp\|','',regex=True)
data5 = data5.rename(columns={'Protein IDs':'Processed Protein IDs'})
data5.set_index('Processed Protein IDs',inplace =True)
data5 = data5[data5.index.str.endswith('HUMAN')]
data5.columns = data5.columns + '_SCP47730' + '_PleuralEffusion' + '_Human' + '_U937'
data5

,Intensity_SCP47730_PleuralEffusion_Human_U937,Intensity 1__161006A_NC_set%2337A_250ms_N3_LC2_exclusionOld_SCP47730_PleuralEffusion_Human_U937,Intensity 2__170211LMRSAM00095_set47B_120min_SCP47730_PleuralEffusion_Human_U937,Intensity 3__170303LMRSAM00093_NC_set%2350H_120min_50ms_SCP47730_PleuralEffusion_Human_U937
Processed Protein IDs,,,,
P13646|K1C13_HUMAN,1.352700e+07,0.0,1.352700e+07,0.0
P02533|K1C14_HUMAN,1.617100e+09,94815000.0,1.522300e+09,0.0
P02538|K2C6A_HUMAN,7.924100e+08,37179000.0,7.552300e+08,0.0
P02768|ALBU_HUMAN,1.521900e+08,0.0,2.602800e+07,126160000.0
P08779|K1C16_HUMAN,3.293200e+08,42257000.0,2.870600e+08,0.0
...,...,...,...,...
Q9Y617|SERC_HUMAN,1.031700e+08,0.0,3.293800e+07,70235000.0
Q9Y678|COPG1_HUMAN,3.306000e+07,0.0,0.000000e+00,33060000.0
Q9Y6E2|5MP1_HUMAN,9.690900e+07,0.0,0.000000e+00,96909000.0


In [55]:
folder_path = "MS-based-SCP/Pleural effusion/SCP61852/"
files = [f for f in os.listdir(folder_path) if f.startswith("all_intensity_included_proteinGroups") and f.endswith(".csv")]
combined_df = pd.DataFrame()
for file in files:
    file_path = os.path.join(folder_path, file)
    temp = pd.read_csv(file_path)  
    columns = ['Protein IDs','Intensity']
    temp = temp[columns]
    temp = process_raw_data(temp)
    temp = temp.merge(
        index_map[['Protein IDs','Processed Protein IDs']],
        on = 'Protein IDs',
        how = 'left'
    )
    temp.drop('Protein IDs',axis = 1,inplace = True)
    temp.set_index('Processed Protein IDs',inplace = True)
    match = re.search(r"_([A-Za-z0-9]+)\.txt", file_path)
    extracted = match.group(1)
    temp.columns = temp.columns + '_' + extracted
    combined_df = pd.concat([combined_df, temp], ignore_index=True)  
combined_df
# file_pattern = 'MS-based-SCP/Pleural effusion/SCP61852/all_intensity_included_proteinGroups_FP18.txt.csv'
# file_list = glob.glob(file_pattern)
# all_data = []
# for file in file_list:
#     dates = re.search(r'(\d{4}_\d{2}_\d{2}_SC\d{2})', file).group(0)
#     data1 = pd.read_csv(file, index_col=0)
#     columns = ['Protein IDs','Intensity']
#     data1 = data1[columns]
#     data1 = process_raw_data(data1)
#     data1['Processed Protein IDs'] = data1['Protein IDs'].str.replace(r'^sp\|', '', regex=True)
#     data1.drop('Protein IDs', axis=1, inplace=True)
#     data1 = data1[[col for col in data1.columns if 'LFQ' not in col]]
#     data1.set_index('Processed Protein IDs', inplace=True)
#     data1.columns = data1.columns + '_' + dates
#     all_data.append(data1)
# final_data = pd.concat(all_data, axis=1)
# final_data.columns = final_data.columns + '_SCP43381' + '_Brain' + '_Mouse' + '_DopaminergicNeurons'

,Intensity_FP18,Intensity_IFN6H,Intensity_SQC67,Intensity_SQC68,Intensity_SQC69,Intensity_SQC71,Intensity_SQC72,Intensity_SQC73,Intensity_SQC74,Intensity_SQC75,...,Intensity_SQC83,Intensity_SQC84,Intensity_SQC85,Intensity_SQC86,Intensity_SQC87,Intensity_SQC95,Intensity_SQC96,Intensity_SQC97,Intensity_SQC98,Intensity_SQC99
0,1.793300e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.562800e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.422200e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.294200e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.955300e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11407000.0
75183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
75184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
75185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
